In [1]:
import sysconfig
import os
import numpy as np
import pandas as pd
import json
import csv
import distutils
import scrapy
import datetime
import requests
import json
import logging
import string
import re
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from requests import Request
from math import sin, cos, sqrt, atan2, radians
from decimal import Decimal
import math

# approximate radius of earth in km
R = 6373.0



In [2]:
class Stayz_Listing(scrapy.Item):
    property_id = scrapy.Field()
    lat = scrapy.Field()
    lng = scrapy.Field()
    init_price = scrapy.Field()
    guests = scrapy.Field()
    heading = scrapy.Field()
    description_full = scrapy.Field()
    url = scrapy.Field()
    bedrooms = scrapy.Field()
    beds = scrapy.Field()
    bathrooms = scrapy.Field()
    property_type = scrapy.Field()
    reviews = scrapy.Field()
    #rating = scrapy.Field()
    #suburb = scrapy.Field()
    #postcode = scrapy.Field()
    #google_addr = scrapy.Field()
    #house_specs = scrapy.Field()
    scraped_date = scrapy.Field()
    syd_brg_deg = scrapy.Field()
    syd_brg = scrapy.Field()
    syd_dist_km = scrapy.Field()
    
        

In [3]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

csv_file = open('/Users/taj/GitHub/scraping/stayz_analysis/WebData/stayz_nsw_extract_' + date_str + '.csv', 'w')

csvw = csv.writer(csv_file
                 ,delimiter=','
                 ,quotechar='"'
                 ,quoting=csv.QUOTE_MINIMAL)

csvw.writerow(["property_id", "lat", "lng", "guests", "init_price"])


39

In [4]:
# http://www.thecodeknight.com/post_categories/search/posts/scrapy_python

class StayzSpider(scrapy.Spider):
    name = 'stayz_crawler'
    

    
    # Full scrape - NSW
    start_urls = ['https://www.stayz.com.au/accommodation/nsw']
    
    # Forbes - has 3 properties on one listing page
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/forbes']
    
    # Orange - has 122 properties on 3 listing pages
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/orange/']
    
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/explorer-country/mudgee'
    #             ,'https://www.stayz.com.au/accommodation/nsw/explorer-country/bathurst'
    #             ,'https://www.stayz.com.au/accommodation/nsw/north-coast/coffs-harbour'
    #             ,'https://www.stayz.com.au/accommodation/nsw/north-coast/port-macquarie']
    
    #start_urls = ['https://www.stayz.com.au/accommodation/nsw/central-coast/gosford/'
    #              ,'https://www.stayz.com.au/accommodation/nsw/explorer-country/orange/'
    #             ]
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        #'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, #Used for pipeline 1
        'FEED_FORMAT':'json', # Used for pipeline 2
        'FEED_URI': '/Users/taj/GitHub/scraping/stayz_analysis/WebData/stayz_nsw_extract_' + datetime.datetime.now().strftime("%Y-%m-%d") + '.json' #Used for pipeline 2
    }
    
    url_pages = []
    
    def calculate_initial_compass_bearing(self, pointA, pointB):
        """
        Calculates the bearing between two points.
        The formulae used is the following:
            θ = atan2(sin(Δlong).cos(lat2),
                      cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
        :Parameters:
          - `pointA: The tuple representing the latitude/longitude for the
            first point. Latitude and longitude must be in decimal degrees
          - `pointB: The tuple representing the latitude/longitude for the
            second point. Latitude and longitude must be in decimal degrees
        :Returns:
          The bearing in degrees
        :Returns Type:
          float
        """
        if (type(pointA) != tuple) or (type(pointB) != tuple):
            raise TypeError("Only tuples are supported as arguments")

        lat1 = math.radians(pointA[0])
        lat2 = math.radians(pointB[0])
        
        lon1 = math.radians(pointA[1])
        lon2 = math.radians(pointB[1])

        diffLong = math.radians(pointB[1] - pointA[1])

        x = math.sin(diffLong) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
                * math.cos(lat2) * math.cos(diffLong))

        initial_bearing = math.atan2(x, y)

        # Now we have the initial bearing but math.atan2 return values
        # from -180° to + 180° which is not what we want for a compass bearing
        # The solution is to normalize the initial bearing as shown below
        initial_bearing = math.degrees(initial_bearing)
        compass_bearing = (initial_bearing + 360) % 360
        
        
        # Calculate the straight line distance in km
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        syd_distance_km = int(R * c)
        
        
        if compass_bearing >= 0.00 and compass_bearing < 22.50:
            compass_dir = 'N'
        elif compass_bearing >= 22.50 and compass_bearing < 67.5:
            compass_dir = 'NE'
        elif compass_bearing >= 67.5 and compass_bearing < 112.50:
            compass_dir = 'E'
        elif compass_bearing >= 112.5 and compass_bearing < 157.50:
            compass_dir = 'SE'
        elif compass_bearing >= 157.5 and compass_bearing < 202.50:
            compass_dir = 'S'
        elif compass_bearing >= 202.5 and compass_bearing < 247.50:
            compass_dir = 'SW'
        elif compass_bearing >= 247.5 and compass_bearing < 292.50:
            compass_dir = 'W'
        elif compass_bearing >= 292.5 and compass_bearing < 337.50:
            compass_dir = 'NW'
        elif compass_bearing >= 337.5 and compass_bearing <= 360.00:
            compass_dir = 'N'
        else:
            compass_dir = 'UN'

        return (int(compass_bearing), compass_dir, int(syd_distance_km))
    
    def parse(self, response):
        
        sel = Selector(response)
        
             
        urls = sel.xpath('//section[@class="c-search-results__main"]/div/article/div/div/div/h3/a/@href').extract()
                
        for u in urls:
        
            request = scrapy.Request('https://www.stayz.com.au/'+ u, callback=self.parse_results )
            
            yield request
            
        # Get the link to the 'Next' page of listings
        url_pages = sel.xpath('/html/body/div/main/div/section/div/nav/ul/li/a/@href').extract()
        
        if len(url_pages) > 0:
            nextPage = url_pages[-1]
        
            nextPageURL = 'https://www.stayz.com.au/'+ nextPage
        
            print("Next URL: " + str(nextPage))
        
            yield(scrapy.Request(nextPageURL, callback=self.parse))
            
    def strip_whitespace(self, st):
    
        st1 = re.sub('\n','',st.string() )
        st2 = re.sub(' +',' ', st1.string() )
        return st2    
                
    def parse_results(self, response):
        
        # Property ID - unique for each property
        p_id1 = response.selector.xpath('//article/header/ol/li/span/span/text()').extract_first()
        p_id2 = re.sub('\n','',p_id1)
        p_id3 = re.sub(' +',' ', p_id2)
        p_id4 = re.search('\d+',p_id3)
        p_id = p_id4.group(0)

        # Geographic location - not exact location, within a block of actual
        # Error handling - if no lat/lon specified place in middle of Botany Harbour
        p_lat = response.selector.xpath('//*[@class="c-map__container"]/@lat').extract_first()
        if p_lat is None:
            p_lat = '-33.990'
        p_lng = response.selector.xpath('//*[@class="c-map__container"]/@lng').extract_first() 
        if p_lng is None:
            p_lng = '151.180'

        #print("Distance to Sydney: {0} km" .format(p_syd_distance_km))
        
        p_compass_bearing, p_dir, p_syd_distance_km = self.calculate_initial_compass_bearing((Decimal(-33.990),Decimal(151.180)),(Decimal(p_lat),Decimal(p_lng)))
        #print("Distance: {2} Compass: {0} Heading: {1}" .format(p_compass_bearing, p_dir, p_syd_distance_km))
        

        # House configuration - number of guests
        p_configuration = response.css(".c-facet__label.u-display--block::text").extract_first()
        p_configuration = re.sub('\n','',p_configuration)
        p_configuration = re.sub(' +',' ', p_configuration)
        p_group = re.search('\d+', p_configuration)
        p_guests = p_group.group(0)
        
        p_heading = response.css(".c-heading--brand.u-h2::text").extract_first()
        
        # Property type from the sub-heading - 'House, Mudgee'
        p_pt_1 = response.selector.xpath('///html/body/main/section/div/article/header/p/small/text()').extract_first()
        
        if p_pt_1 is not None:
            p_pt_1 = re.sub('\n','', p_pt_1)
            p_pt_1 = re.sub(' +',' ', p_pt_1)
            if re.search('B&B', p_pt_1):
                p_property_type = 'B&B'
            else:
                p_pt_2 = re.search('\w+',p_pt_1)
                p_property_type = p_pt_2.group(0)
        else:
            p_property_type = 'Unknown'
        
        p_house_specs = response.selector.xpath('//div[@class="c-facets c-facets--inline"]/div/span/span/text()').extract()
        
        # Initialize variables
        n_bedrooms = 0
        n_beds = 0
        n_bathrooms = 0
        
        for j in p_house_specs:
            if re.search('bedroom',j):
                # Extract number of guests
                n_br = re.search('\d+',j)
                n_bedrooms = n_br.group(0)
                
            if re.search('bed[s]?$',j):
                # Extract number of guests
                n_bd = re.search('\d+',j)
                n_beds = n_bd.group(0)
                
            if re.search('bathroom',j):
                # Extract number of guests
                n_bath = re.search('\d+',j)
                n_bathrooms = n_bath.group(0)
                
                
        # The basic price - 'From $xxx per night'        
        p_ip_1 = response.selector.xpath('//header[@class="c-quote__header"]/p/span/span/span[@class="u-h1"]/text()').extract_first()
        
        p_init_price = 0
        
        if p_ip_1 is not None:
            p_ip_3 = p_ip_1.replace(',','')
            p_ip_4 = re.search('\d+',p_ip_3)
            p_init_price = p_ip_4.group(0)
        else:
            p_init_price = 0
        
        # Ratings and reviews
        p_rating = response.selector.xpath('//header[@class="c-reviews__header"]/span/span/span[@class="c-facet__label"]/span/text()').extract_first()
        
        p_r1 = response.selector.xpath('//html/body/main/section/div/article/header/p[2]/span/span[2]/small/text()').extract_first()
        
        # Initialise
        p_reviews = 0
        
        if p_r1 is not None:
            p_r2 = re.sub('\n','',p_r1)
            p_r2 = re.sub(' +',' ',p_r2)
            p_r3 = re.search('\d+', p_r2)
            p_reviews = p_r3.group(0)
        else:
            p_reviews = 0
        
        # Full text of property details
        p_description_full = response.xpath('//div[@id="property-description-body"]/p/text()').extract_first()
        
        # Reverse geocoding on Google API for street address:
        latlng = "" + p_lat + "," + p_lng 
        
        print("Scanning Property ID: " + p_id + "-" + response.url)
        
        p = Stayz_Listing()
        p['property_id'] = p_id
        p['lat'] = p_lat
        p['lng'] = p_lng
        p['guests'] = p_guests
        p['heading'] = p_heading
        p['url'] = response.url
        p['init_price'] = p_init_price
        p['bedrooms'] = n_bedrooms
        p['beds'] = n_beds
        p['bathrooms'] = n_bathrooms
        p['description_full'] = p_description_full
        p['property_type'] = p_property_type
        #p['rating'] = p_rating
        p['reviews'] = p_reviews
        #p['suburb'] = p_suburb
        #p['postcode'] = p_postcode
        #p['house_specs'] = p_house_specs
        #p['google_addr'] = google_addr.text
        p['syd_dist_km'] = p_syd_distance_km
        p['syd_brg_deg'] = p_compass_bearing
        p['syd_brg'] = p_dir
        p['scraped_date'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        
        csvw.writerow([p_id,p_lat,p_lng,p_guests,p_init_price])

        
        yield p        


In [5]:
process = CrawlerProcess({ 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)' })

process.crawl(StayzSpider) 
process.start()

csv_file.close()

2018-03-10 14:24:01 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-03-10 14:24:01 [scrapy.utils.log] INFO: Versions: lxml 4.1.0.0, libxml2 2.9.4, cssselect 1.0.3, parsel 1.3.1, w3lib 1.18.0, Twisted 16.6.0, Python 3.6.3 |Anaconda custom (64-bit)| (default, Oct  6 2017, 12:04:38) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 17.2.0 (OpenSSL 1.0.2n  7 Dec 2017), cryptography 2.0.3, Platform Darwin-17.4.0-x86_64-i386-64bit
2018-03-10 14:24:01 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': '/Users/taj/GitHub/scraping/stayz_analysis/WebData/stayz_nsw_extract_2018-03-10.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


Next URL: /accommodation/nsw?page=2
Scanning Property ID: 9080772-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/jindabyne/9080772
Scanning Property ID: 9096264-https://www.stayz.com.au//accommodation/nsw/south-coast/currarong/9096264
Scanning Property ID: 9151070-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook/9151070
Scanning Property ID: 9012574-https://www.stayz.com.au//accommodation/nsw/hunter/hunter-valley/9012574
Scanning Property ID: 9185654-https://www.stayz.com.au//accommodation/nsw/hunter/newcastle/9185654
Scanning Property ID: 9007287-https://www.stayz.com.au//accommodation/nsw/south-coast/jervis-bay/9007287
Scanning Property ID: 9016485-https://www.stayz.com.au//accommodation/nsw/north-coast/coffs-harbour/9016485
Scanning Property ID: 9082905-https://www.stayz.com.au//accommodation/nsw/south-coast/bermagui/9082905
Scanning Property ID: 9132136-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook/9132136
Scanning Property ID: 9179

Scanning Property ID: 9076897-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/byron-bay/9076897
Scanning Property ID: 9153438-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/jindabyne/9153438
Next URL: /accommodation/nsw?page=5
Scanning Property ID: 9117016-https://www.stayz.com.au//accommodation/nsw/illawarra/austinmer/9117016
Scanning Property ID: 9028328-https://www.stayz.com.au//accommodation/nsw/south-coast/narooma/9028328
Scanning Property ID: 9187871-https://www.stayz.com.au//accommodation/nsw/south-coast/kiama/9187871
Scanning Property ID: 9174094-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/byron-bay/9174094
Scanning Property ID: 9144581-https://www.stayz.com.au//accommodation/nsw/south-coast/mollymook-beach/9144581
Scanning Property ID: 9004577-https://www.stayz.com.au//accommodation/nsw/blue-mountains/blackheath/9004577
Scanning Property ID: 9095715-https://www.stayz.com.au//accommodation/nsw/south-coast/currarong/90

Scanning Property ID: 9033916-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/kingscliff/9033916
Scanning Property ID: 9173402-https://www.stayz.com.au//accommodation/nsw/south-coast/gerringong/9173402
Scanning Property ID: 9181151-https://www.stayz.com.au//accommodation/nsw/new-england/armidale/9181151
Scanning Property ID: 9024954-https://www.stayz.com.au//accommodation/nsw/illawarra/bundeena/9024954
Scanning Property ID: 9142052-https://www.stayz.com.au//accommodation/nsw/north-coast/sawtell/9142052
Scanning Property ID: 9102787-https://www.stayz.com.au//accommodation/nsw/south-coast/currarong/9102787
Scanning Property ID: 9026342-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/byron-bay/9026342
Scanning Property ID: 9015535-https://www.stayz.com.au//accommodation/nsw/hunter/pokolbin/9015535
Scanning Property ID: 9077089-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/byron-bay/9077089
Scanning Property ID: 9117059-ht

Scanning Property ID: 9085136-https://www.stayz.com.au//accommodation/nsw/southern-highlands/goulburn/9085136
Scanning Property ID: 9206113-https://www.stayz.com.au//accommodation/nsw/sydney/bondi-beach/9206113
Scanning Property ID: 9147610-https://www.stayz.com.au//accommodation/nsw/blue-mountains/wentworth-falls/9147610
Scanning Property ID: 9016898-https://www.stayz.com.au//accommodation/nsw/hunter/cessnock/9016898
Scanning Property ID: 9190393-https://www.stayz.com.au//accommodation/nsw/hunter/east-gresford/9190393
Scanning Property ID: 9148674-https://www.stayz.com.au//accommodation/nsw/explorer-country/orange/9148674
Scanning Property ID: 9020305-https://www.stayz.com.au//accommodation/nsw/south-coast/berry/9020305
Scanning Property ID: 9151420-https://www.stayz.com.au//accommodation/nsw/blue-mountains/wentworth-falls/9151420
Scanning Property ID: 9023576-https://www.stayz.com.au//accommodation/nsw/central-coast/macmasters-beach/9023576
Scanning Property ID: 9021380-https://www.s

...

Scanning Property ID: 9170946-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/farrants-hill/9170946
Scanning Property ID: 9019293-https://www.stayz.com.au//accommodation/nsw/south-coast/lake-conjola/9019293
Scanning Property ID: 9191236-https://www.stayz.com.au//accommodation/nsw/explorer-country/mudgee/9191236
Scanning Property ID: 9057997-https://www.stayz.com.au//accommodation/nsw/southern-highlands/bundanoon/9057997
Scanning Property ID: 9062923-https://www.stayz.com.au//accommodation/nsw/north-coast/smiths-lake/9062923
Scanning Property ID: 9147356-https://www.stayz.com.au//accommodation/nsw/blue-mountains/katoomba/9147356
Scanning Property ID: 9022677-https://www.stayz.com.au//accommodation/nsw/central-coast/copacabana/9022677
Scanning Property ID: 9014469-https://www.stayz.com.au//accommodation/nsw/south-coast/berry/9014469
Scanning Property ID: 9181554-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/jindabyne/9181554
Scanning Property ID: 91632

Scanning Property ID: 9064234-https://www.stayz.com.au//accommodation/nsw/hunter/pokolbin/9064234
Scanning Property ID: 9206929-https://www.stayz.com.au//accommodation/nsw/north-coast/bellingen/9206929
Scanning Property ID: 9187340-https://www.stayz.com.au//accommodation/nsw/south-coast/sussex-inlet/9187340
Scanning Property ID: 9085252-https://www.stayz.com.au//accommodation/nsw/southern-highlands/bowral/9085252
Scanning Property ID: 9190384-https://www.stayz.com.au//accommodation/nsw/south-coast/merimbula/9190384
Scanning Property ID: 9193336-https://www.stayz.com.au//accommodation/nsw/south-coast/jervis-bay/9193336
Scanning Property ID: 9053677-https://www.stayz.com.au//accommodation/nsw/south-coast/batemans-bay/9053677
Scanning Property ID: 9029438-https://www.stayz.com.au//accommodation/nsw/south-coast/currarong/9029438
Scanning Property ID: 9145492-https://www.stayz.com.au//accommodation/nsw/north-coast/port-macquarie/9145492
Scanning Property ID: 9021387-https://www.stayz.com.au

Scanning Property ID: 9172109-https://www.stayz.com.au//accommodation/nsw/south-coast/culburra-beach/9172109
Scanning Property ID: 9215286-https://www.stayz.com.au//accommodation/nsw/explorer-country/mudgee/9215286
Scanning Property ID: 9153200-https://www.stayz.com.au//accommodation/nsw/explorer-country/coonabarabran/9153200
Scanning Property ID: 9185889-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/wooli/9185889
Scanning Property ID: 9208335-https://www.stayz.com.au//accommodation/nsw/southern-highlands/bowral/9208335
Scanning Property ID: 9028857-https://www.stayz.com.au//accommodation/nsw/central-coast/budgewoi/9028857
Scanning Property ID: 9099235-https://www.stayz.com.au//accommodation/nsw/south-coast/vincentia/9099235
Scanning Property ID: 9169092-https://www.stayz.com.au//accommodation/nsw/south-coast/tuross-head/9169092
Scanning Property ID: 9187720-https://www.stayz.com.au//accommodation/nsw/north-coast/forster/9187720
Scanning Property ID: 9025027-htt

Scanning Property ID: 9059575-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/byron-bay/9059575
Scanning Property ID: 9027568-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/kingscliff/9027568
Scanning Property ID: 9083979-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/kingscliff/9083979
Scanning Property ID: 9152089-https://www.stayz.com.au//accommodation/nsw/south-coast/kangaroo-valley/9152089
Scanning Property ID: 9216831-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/byron-bay/9216831
Scanning Property ID: 9109089-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/ballina/9109089
Scanning Property ID: 9021024-https://www.stayz.com.au//accommodation/nsw/south-coast/pambula-beach/9021024
Scanning Property ID: 9181518-https://www.stayz.com.au//accommodation/nsw/south-coast/berry/9181518
Scanning Property ID: 9153367-https://www.stayz.com.au//accommodation/nsw/north-coast/coffs-

Scanning Property ID: 9132090-https://www.stayz.com.au//accommodation/nsw/south-coast/kangaroo-valley/9132090
Scanning Property ID: 9189174-https://www.stayz.com.au//accommodation/nsw/south-coast/kangaroo-valley/9189174
Scanning Property ID: 9007262-https://www.stayz.com.au//accommodation/nsw/central-coast/pearl-beach/9007262
Scanning Property ID: 9210965-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/jindabyne/9210965
Scanning Property ID: 9164768-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/yamba/9164768
Scanning Property ID: 9151234-https://www.stayz.com.au//accommodation/nsw/south-coast/jervis-bay/9151234
Scanning Property ID: 9060103-https://www.stayz.com.au//accommodation/nsw/northern-rivers-byron-bay/kingscliff/9060103
Scanning Property ID: 9188751-https://www.stayz.com.au//accommodation/nsw/south-coast/kiama/9188751
Scanning Property ID: 9110247-https://www.stayz.com.au//accommodation/nsw/snowy-mountains/jindabyne/9110247
Scanning Property 